# Practical Natural Language Processing (NLP)
adapted from *COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/* 

In this lecture, we'll do some practical NLP following up on last week's theoretical lecture. We will do some basic text processing followed by a sentiment analysis for movie reviews. For this purpose, we'll introduce  the [Natural Language Toolkit (NLTK)](http://www.nltk.org/), a Python library for  Natural Language Processing. 

We won't cover NLTK or NLP extensively here – this lecture is meant to give you a few pointers if you want to use NLP in the future, e.g., for your project.

**Reading:** 

[S. Bird, E. Klein, and E. Loper, *Natural Language Processing with Python – Analyzing Text with the Natural Language Toolkit*](http://www.nltk.org/book/). 


[C. Manning and H. Schütze, *Foundations of Statistical Natural Language Processing* (1999).](http://nlp.stanford.edu/fsnlp/)

[D. Jurafsky and J. H. Martin, *Speech and Language Processing* (2016).](https://web.stanford.edu/~jurafsky/slp3/)


NLP is a really broad field.  Some really imporant NLP tasks that we mostly won't dive into deeply include: 
* Part of speech tagging (what are the nouns, verbs, adjectives, prepositions).
+ Information Extraction
+ Sentiment Analysis (determine the attitude of text, e.g., is it positive or negative).
+ Semantic Parsing (translate natural language into a formal meaning representation).

Generally, the current state-of-the-art for many NLP tasks is to find a good way to represent the text ("extract features") and then to use machine learning / statistics tools, such as classification or clustering. 

Our goal today is to use NLTK + scikit-learn to do some basic NLP tasks.  

### Install datasets and models

To use NLTK, you must first download and install the datasets and models. 

Run `/usr/local/bin/pip3 install nltk` (or whatever setup works for you to install packages)

Then execute the following cell:

In [ ]:
import nltk
nltk.download('all')

In [ ]:
# imports and setup
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')

## Basics of NLTK

We have downloaded a set of text corpora above. Here is a list of these texts:

In [ ]:
from nltk.book import *

Let's look at the first 20 words of that text:

In [ ]:
moby      = text1
sense     = text2
genesis   = text3
inaugural = text4
monty     = text6

book_names = [ "Moby", "Sense", "Genesis", "Inaugural", "Chat", "Monty", "WSJ", "Personals", "Thursday" ]

In [ ]:
moby[0:50]

### Text Statistics

We can check the length of a text. The text of Moby Dick is 260,819 words, whereas Monty Python and the Holy Grail has 16,967 words. 

In [ ]:
len( moby )

In [ ]:
len( monty )

We can check for the frequency of a word. The word "swallow" appears 10 times in Monty Python.

In [ ]:
monty.count( "swallow" )

We might want to know the context in which "swallow" appears in the text

"You shall know a word by the company it keeps." – John Firth

Use the [`concordance`](http://www.nltk.org/api/nltk.html#nltk.text.Text.concordance) function to print out the words just before and after all occurrences of the word "swallow". 

In [ ]:
monty.concordance( "swallow" )

And if we look for Ishmael in Moby Dick:

In [ ]:
moby.concordance( "Ishmael" )

We can see what other words frequently appear in the same context using the  [`similar`](http://www.nltk.org/api/nltk.html#nltk.text.Text.similar) function.  

In [ ]:
monty.similar( "african" )

This means that 'african' and 'unladen' both appeared in the text with the same word just before and just after. To see what the phrase is, we can use the [`common_contexts`](http://www.nltk.org/api/nltk.html#nltk.text.Text.concordance) function. 

In [ ]:
monty.common_contexts( ["unladen", "african"] )

We see that both "an unladen swallow" and "an african swallow" appear in the text. 

In [ ]:
monty.concordance( "unladen" )
print()
monty.concordance( "african" )

### Dispersion plot

`text4` is the Inaugural Address Corpus which includes inaugural addresses going back to 1789. 
We can use a dispersion plot to see where in a text certain words appear, and hence how the language of the address has changed over time. 


In [ ]:
inaugural.dispersion_plot( ["citizens", "democracy", "freedom", "duty", "America", "nation", "God"] )

In [ ]:
inaugural[:100]

In [ ]:
inaugural[-100:]

### Exploring texts using statistics

We'll explore a text by counting the frequency of different words.

The total number of words ("outcomes") in Moby Dick is 260,819 and the number of different words is 19,317. 

In [ ]:
frequency_dist = FreqDist( moby )
print( frequency_dist )

# Find 50 most common words
print( '\n', frequency_dist.most_common(50) )

# Not suprisingly, whale occurs quite frequently (906 times!)
print( '\n', frequency_dist['whale'] )

We can find all the words in Moby Dick with more than 15 characters

In [ ]:
unique_words = set( moby )
long_words = [ w for w in unique_words if len(w) > 15 ]
long_words

### Stopword Removal

Sometimes, it is useful to ignore frequently used words, to concentrate on the meaning of the remaining words. These are referred to as *stopwords*. Examples are "the", "was", "is", etc. 

NLTK comes with a stopword corpus. 

In [ ]:
from nltk.corpus import stopwords

stopwords = nltk.corpus.stopwords.words( 'english' )
print( stopwords )

Depending on the task, these stopwords are important modifiers, or superfluous content. 

### Exercise 1.1: Frequent Words
Find the most frequently used words in Moby Dick that are not stopwords and not punctuation. Hint: [`str.isalpha()`](https://docs.python.org/3/library/stdtypes.html#str.isalpha) could be useful here.

In [ ]:
fd = FreqDist( moby )

[ w for w in fd.most_common(150) if w[0].lower() not in stopwords and w[0][0].isalpha() ]
#   len([l for l in w[0] if l.isalpha()]) > 0

### Stopwords in Different Corpora
Is there a difference between the frequency in which stopwords appear in the different texts? 

In [ ]:
def content_fraction( text ):
    stopwords = nltk.corpus.stopwords.words( 'english' )
    content = [ w for w in text if w.lower() not in stopwords ]
    return len( content ) / len( text )

for i,t in enumerate( [text1,text2,text3,text4,text5,text6,text7,text8,text9] ):
    print( "%10s - Content percent: %.1f%%" % (book_names[i], content_fraction(t)*100) )

Apparently, "Personals Corpus" has the most content. Monty Python has more content then Moby Dick...

### Collocations
A *collocation* is a sequence of words that occur together unusually often, we can retreive these using the [`collocations()`](http://www.nltk.org/api/nltk.html#nltk.text.Text.collocations) function.

In [ ]:
moby.collocation_list() # Bi-grams / Two-grams

In [ ]:
print( moby.collocation_list.__doc__ )

In [ ]:
moby.collocation_list( window_size = 3 )

## Sentiment analysis for movie reviews
We ask the simple question: Is the attitude of a movie review positive or negative? 

How can we approach this question?

Our data is a corpus consisting of 2000 movie reviews together with the user's sentiment polarity (positive or negative). Our goal is to predict the sentiment polarity from just the review. 

Of course, this is something that we can do very easily: 
1. That movie was terrible. -> negative
+ That movie was great! -> positive

More information about this dataset is available [from this website](https://www.cs.cornell.edu/people/pabo/movie-review-data/).



In [ ]:
from nltk.corpus import movie_reviews as reviews

The datset contains 1000 positive and 1000 negative movie reviews. 

The paths to / IDs for the individual reviews are accessible via the fileids() call:

In [ ]:
reviews.fileids()[0:5]

We can access the positives or negatives explicitly:

In [ ]:
len( reviews.fileids('pos') )

There are in fact 1000 positive and 1000 negative reviews:

In [ ]:
num_reviews = len( reviews.fileids() )
print( "Number of reviews:", num_reviews )
print( "# Positive: %d, # Negative: %d" % ( len(reviews.fileids('pos')), len(reviews.fileids('neg') ) ) )

Let's see the review for the third movie. Its a negative review for [The Mod Squad](https://www.rottentomatoes.com/m/mod_squad/), which has a "rotten" rating on rotten tomatoes. 

![Mod Squad at Rotten Tomatoes](mod_squad.png)

In [ ]:
# the name of the file 
fid = reviews.fileids()[ 2 ]
print( fid )

print( '\n', reviews.raw(fid) )

print( '\n', reviews.categories(fid) )

print( '\n', reviews.words(fid) )

Let's look at some sentences that indicate that this is a negative review:

 * "it is movies like these that make a jaded movie viewer thankful for the invention of the timex indiglo watch"
 * "sounds like a cool movie , does it not ? after the first fifteen minutes , it quickly becomes apparent that it is not ." 
 * "nothing spectacular"
 * "avoid this film at all costs"

### A Custom Algorithm
We'll build a sentiment classifier using methods we already know to predicts the label ['neg', 'pos'] from the review text

`reviews.categories( file_id )` returns the label ['neg', 'pos'] for that movie

In [ ]:
categories = [ reviews.categories(fid) for fid in reviews.fileids() ] # 'pos' or 'neg'
labels = { 'pos':1, 'neg':0 }

# Create the labels - 1 for positive, 0 for negative
y = [ labels[ x[0] ] for x in categories ]

display( y[:10] )
display( y[-10:] )

display( len(y) )

Here, we collect all words into a nested array datastructure:

In [ ]:
doc_words = [ list(reviews.words(fid)) for fid in reviews.fileids() ]

In [ ]:
# First 10 words of the third review:
doc_words[2][1:10]

Here we get all of the words in the reviews and make a FreqDist, pick the most common 2000 words and remove the stopwords.

In [ ]:
# get the 2000 most common words in lowercase
most_common = nltk.FreqDist( w.lower() for w in reviews.words() ).most_common( 2000 )

# Remove stopwords
filtered_words = [word_tuple for word_tuple in most_common if word_tuple[0].lower() not in stopwords]

# Remove punctuation marks
filtered_words = [word_tuple for word_tuple in filtered_words if word_tuple[0].isalpha()]

print( "Number of filtered words:", len(filtered_words) )
filtered_words[0:50]

We  extract this word list from the frequency tuple.

In [ ]:
word_features = [ word_tuple[0] for word_tuple in filtered_words ] # Keep the word, ignore the frequency

word_features[:10]

We define a function that takes a document and returns a list of zeros and ones indicating which of the words in  `word_features` appears in that document. 

In [ ]:
def document_features( document ):
    document_words = set( document ) # Throw away duplicates by making a set
    features = np.zeros( len( word_features ) ) # Start with a vector of 0's
    for i, word in enumerate( word_features ):
        features[i] = ( word in document_words )
    return features

In [ ]:
mpFeatures = document_features( monty ) # Using Monty Python as example, but not useful with respect to analyzing a review.
print( mpFeatures )
# 1, 1, 0 -> includes "film" and "one", but not "movie"

Let's just focus on the third document. Which words from `word_features` are in this document? 

In [ ]:
words_in_doc_2 = document_features( doc_words[2] )
print( words_in_doc_2 )

inds = np.where( words_in_doc_2 == 1 )[0]
print( '\n', [word_features[i] for i in inds] )

Now we build our feature set for all the reviews.

In [ ]:
X = np.zeros( [num_reviews, len(word_features)] )

for i in range( num_reviews ):
    X[i,:] = document_features( doc_words[i] )

X[0:5]

In [ ]:
# X => 2000 by 1821 (# of Reviews by # of Features (words))
X.shape

The result is a feature vector for each of these reviews that we can use in classification.

Now that we have features for each document and labels, **we have a classification problem!** 

NLTK has a built-in classifier, but we'll use the scikit-learn classifiers we're already familiar with. 

Let's try k-nearest neighbors:

In [ ]:
k = 40
model = KNeighborsClassifier( n_neighbors=k )
scores = cross_val_score( model, X, y, cv=10 )

print( scores )

# Is ~50% success good?  Why?  60% was good for digits...

And SVM:

In [ ]:
model = svm.SVC( kernel='rbf', C=20 )
scores = cross_val_score( model, X, y, cv=3 )

print( scores )

Here we can see that kNN with these parameters is less accurate than SVM, which is about 80% accurate. Of course, we could now use the cross validation scores to find the optimal parameters, `k` and `C`

So, let's see what our algorithm things about the Mod Squad! 

In [ ]:
XTrain, XTest, yTrain, yTest = train_test_split( X, y, random_state=1, test_size=0.2 )

_ = model.fit( XTrain, yTrain )

In [ ]:
mod_squad = [ X[2] ]
mod_squad # <- Display the features (words) in the Mod Squad review...

In [ ]:
model.predict( mod_squad ) # Returns the label... in this case 0 for negative, 1 for positive review

Our model says 0 - so a bad review! We have successfully built a classifier that can detect the Mod Squad review as a bad review! 

In [ ]:
# What if we treat monty python's script as a movie review?
#   - Perhaps not a useful example, but kind of a fun though experiment.
mpRow = [mpFeatures]
print( model.predict( mpRow ) )

Let's take a look at a mis-classified movie. Remember, that the first 1000 movies are negative reviews, so we can just look for the first negative one:

In [ ]:
model.predict( X[0:10] ) # Look at the first 10 reviews

Review 9, which was misclassified, is for Aberdeen, which has [generally favorable reviews](https://www.rottentomatoes.com/m/aberdeen/). Let's looks at the review:

In [ ]:
fid = reviews.fileids()[8]

print('\n', reviews.raw(fid))
print('\n', reviews.categories(fid) )

So if we read this, we can see that this is a negative review, but not a terrible review. Take this sentence for example: 

 * "if signs & wonders sometimes feels overloaded with ideas , at least it's willing to stretch beyond what we've come to expect from traditional drama"
 * "yet this ever-reliable swedish actor adds depth and significance to the otherwise plodding and forgettable aberdeen , a sentimental and painfully mundane european drama"

## We could have also used the Classifier from the NLTK library

Below is the sentiment analysis from [Ch. 6 of the NLTK book](http://www.nltk.org/book/ch06.html). 



In [ ]:
documents = [ (list(reviews.words(fileid)), category)
                    for category in reviews.categories() 
                        for fileid in reviews.fileids(category) ]

Extract the features from all of the documents

In [ ]:
def document_features(document):    
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d), c) for (d,c) in documents]

Split into train_set, test_set and perform classification 

In [ ]:
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print(nltk.classify.accuracy(classifier, test_set))

classifier.show_most_informative_features(30)

NLTK gives us 88% accuracy, which isn't bad, but our home-made naive algorithm also achieved a respectable 80%.


What improvements could we have made? Obviously, we could have used more data, or – in our home-grown model select words that discriminate between good and bad reviews. We could have used n-grams, e.g., to catch "not bad" as a postitive sentiment.

In [ ]:
from collections import Counter

def document_features2(document):
    counter = Counter(document)
    document_words = set(document)
    features = np.zeros(len(word_features))
    for i, word in enumerate(word_features):
        features[i] = counter[word]
    return features

X2 = np.zeros([num_reviews,len(word_features)])
for i in range(num_reviews):
    X2[i,:] = document_features2(doc_words[i])

X2[0:5]

In [ ]:
model = svm.SVC(kernel='rbf', C=20)
scores = cross_val_score(model, X2, y, cv=3)
print(scores)

This is slightly worse than we did with the "bag of words" model.  The number of occurences of words seems to not be particulary useful here

In [ ]:
from sklearn import tree
from sklearn import metrics

In [ ]:
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1, test_size=0.2)
decisionTree = tree.DecisionTreeClassifier(max_depth=3)

decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))
tree.plot_tree(decisionTree, filled=True, feature_names=word_features);

In [ ]:
decisionTree = tree.DecisionTreeClassifier(max_depth=5)

decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

In [ ]:
XTrain, XTest, yTrain, yTest = train_test_split(X2, y, random_state=1, test_size=0.2)
decisionTree = tree.DecisionTreeClassifier(max_depth=5)

decisionTree = decisionTree.fit(XTrain, yTrain)

y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))

y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

In [ ]:
from sklearn.decomposition import PCA 
pca_model = PCA()
X_PCA = pca_model.fit_transform( X )

# Rotating the axes... 
#  - Initially each axes was a word (well, the presence/absence of the word)
#  - Now each axis is (sort of) a group of words

In [ ]:
def PCASort(col):
    indices = list(range(len(word_features)))
    return sorted(indices, key= lambda x: abs(pca_model.components_[col][x]))

# Determining (according to PCA analysis) which words are the most important
pca0 = PCASort(0)
for x in pca0[-25:]:
    print( "%s: %.1f%%" % (word_features[x], pca_model.components_[0][x]*100 ) )

In [ ]:
pca1 = PCASort(1)
for x in pca1[-25:]:
    print(word_features[x], pca_model.components_[1][x])

In [ ]:
# Perform SVM analysis on the rotated data:

model = svm.SVC( kernel='rbf', C=20 )

scores = cross_val_score(model, X_PCA, y, cv=3)
print( scores )

In [ ]:
print( "Shape:", X_PCA.shape )

for cols in range( 5, 105, 5 ):
    model = svm.SVC( kernel='rbf', C=20 )
    # In the cross validation, we throw away a huge percent of the data.  (Eg: Use 5 columns, 10 columns,
    #         up to 100 columns (out of 1821))
    #    - Remember what PCA has done with each axis...
    #      - Accuracy when using < 10% of data is at same level as using entire data set.
    scores = cross_val_score( model, X_PCA[:, :cols], y, cv=3 )
    print( "Used", cols, "PCA components and got scores", scores )

This shows that once we turn text into "points" (vectors), we don't have to think of it as (direclty) NLP (at least as a first approximation).  We can just use our set of ML regression techniques we have been discussing this semester.